In [ ]:
# input is the video and label result; output is the video with labels writing on it

In [1]:
import numpy as np
import cv2
import os
import uuid
import math
from IPython.core.debugger import Tracer

In [2]:
video_filename = 'archive/data1/video_segmentation_labeled/ea87de4a-ac07-11e6-bfe2-d017c2cf90d9.avi'
fps = 25
frame_num = 20
show_width = 640
show_height = 360
show_video = False

label_filename = video_filename.replace('avi', 'txt')

with open(label_filename) as f:
    content = f.readlines()

cap = cv2.VideoCapture(video_filename)

frame_count = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT))
label_result = [-1] * frame_count
# fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)

internal = 1000 / fps
internal_ary = [int(internal), int(internal * 2), int(internal * 4)]
internal_idx = 0
frame_idx = 0

text = ["none", "normal", "paging", "shake", "else"]
if show_video:
    speed_text = ["1X", "0.5X", "0.25X"]
    while cap.isOpened():
        frame_idx = int(cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES))
        ret, frame = cap.read()
        print frame_idx
        if ret != True:
            break
        frame = cv2.resize(frame, (show_width, show_height), interpolation = cv2.INTER_AREA)
        font = cv2.FONT_HERSHEY_SIMPLEX
        if content[0][frame_idx] != '-':
            cv2.putText(frame, text[int(content[0][frame_idx])], (show_width - 200, show_height - 20), font, 1, (0, 0, 255), 2)
        cv2.putText(frame, str(frame_idx) + '/' + str(frame_count), (10, show_height - 20), font, 1, (0,0,255), 2)
        cv2.putText(frame, speed_text[internal_idx], (10, 30), font, 1, (0, 0, 255), 2)
        cv2.imshow('frame',frame)
        k = cv2.waitKey(internal_ary[internal_idx])
        if k & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        # when pressing 'j', speed up one level
        if k & 0xFF == ord('j'):
            internal_idx = np.max([internal_idx - 1, 0])
        # when pressing 'k', speed down one level
        if k & 0xFF == ord('k'):
            internal_idx = np.min([internal_idx + 1, 2])
        if k & 0xFF == ord('t'):
            r = cv2.waitKey()
            # when pressing 't' again, continue to play
            while r & 0xFF != ord('t'):
                if r & 0xFF == ord('n'):
                    frame_idx = int(cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES))
                    ret, frame = cap.read()
                    frame = cv2.resize(frame, (show_width, show_height), interpolation = cv2.INTER_AREA)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    if content[0][frame_idx] != '-':
                        cv2.putText(frame, text[int(content[0][frame_idx])], (show_width - 200, show_height - 20), font, 1, (0, 0, 255), 2)
                    cv2.putText(frame, str(frame_idx) + '/' + str(frame_count), (10, show_height - 20), font, 1, (0,0,255), 2)
                    cv2.putText(frame, speed_text[internal_idx], (10, 30), font, 1, (0, 0, 255), 2)
                    cv2.imshow('frame', frame)
                # when pressing 'p', go previous frame
                if r & 0xFF == ord('p'):
                    frame_idx = int(cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES))
                    frame_idx = np.max([0, frame_idx - 2])
                    cap.set(1, frame_idx)
                    ret, frame = cap.read()
                    frame = cv2.resize(frame, (show_width, show_height), interpolation = cv2.INTER_AREA)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    if content[0][frame_idx] != '-':
                        cv2.putText(frame, text[int(content[0][frame_idx])], (show_width - 200, show_height - 20), font, 1, (0, 0, 255), 2)
                    cv2.putText(frame, str(frame_idx) + '/' + str(frame_count), (10, show_height - 20), font, 1, (0,0,255), 2)
                    cv2.putText(frame, speed_text[internal_idx], (10, 30), font, 1, (0, 0, 255), 2)
                    cv2.imshow('frame', frame)
                r = cv2.waitKey()
        if k & 0xFF == ord('h'):
            # go back
            frame_idx = int(cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES))
            frame_idx = np.max([0, frame_idx - frame_num])
            cap.set(1, frame_idx)
            ret, frame = cap.read()
            cv2.imshow('frame', frame)
        # when pressing 'l', go forward frame_num frames
        if k & 0xFF == ord('l'):
            # go forward
            frame_idx = int(cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES))
            frame_idx = np.min([frame_count - 1, frame_idx + frame_num])
            cap.set(1, frame_idx)
            ret, frame = cap.read()
            cv2.imshow('frame', frame)
    cap.release()
    cv2.destroyAllWindows()


# save the video with label result:
name = video_filename.replace('.avi', '_out.avi')
font = cv2.FONT_HERSHEY_SIMPLEX
fourcc = cv2.cv.CV_FOURCC(*'XVID')
out = cv2.VideoWriter(name, fourcc, 25, (show_width, show_height))
cap = cv2.VideoCapture(video_filename)
while cap.isOpened():
    frame_idx = int(cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES))
    ret, frame = cap.read()
    if ret != True:
        break
    frame = cv2.resize(frame, (show_width, show_height), interpolation = cv2.INTER_AREA)
    if content[0][frame_idx] != '-':
        cv2.putText(frame, text[int(content[0][frame_idx])], (show_width - 200, show_height - 20), font, 1, (0, 0, 255), 2)
    cv2.putText(frame, str(frame_idx) + '/' + str(frame_count), (10, show_height - 20), font, 1, (0,0,255), 2)
    out.write(frame)
out.release()
cap.release()
cv2.destroyAllWindows()